In [ ]:
import numpy as np
import pagn.constants as ct
import matplotlib.pyplot as plt
import pagn

import NT_disk_Eqns_V1 as jscript

from os import makedirs

import pandas as pd

import Novikov
#some nicer code based on Test_zone.ipynb

In [ ]:
MBH_powers= 4,5,6,7,8

spins=0.9, 0.99
alphas=0.1, 0.01

mdot = 0.1 #in terms of Ledd
eps = 0.1
le = mdot*eps

steps=int(1e4)

for power in MBH_powers:
    for alpha in alphas:
        for spin in spins:
            print(f'MBH=1e{power} MSun, alpha={alpha}, spin={spin}')
            path=f'EMRIs_Jupiter_2/NT_profiles/logMBH{power}/alpha{alpha}/mdot{mdot}/spin{spin}/'

            MBH=10**power*ct.MSun

            disk = Novikov.NovikovThorneAGN(Mbh=MBH, alpha=alpha, spin=spin)
            Rmin = disk.Rmin
            Rmax = disk.Rmax
            disk.solve_disk(steps=steps, path=path, save_to_file=True)
            disk.plot(path=path, save_to_file=True)

In [ ]:
MBH_power=6
alpha=0.1
mdot=0.1
le=0.01
eps=0.1
spin=0.9

MBH=10**MBH_power * ct.MSun #in kgs

path=f'EMRIs_Jupiter_2/NT_profiles/logMBH{MBH_power}/alpha{alpha}/mdot{mdot}/spin{spin}/'

disk = Novikov.NovikovThorneAGN(Mbh=MBH, alpha=alpha)
Rmin = disk.Rmin
Rmax = disk.Rmax
disk.solve_disk(path=path, save_to_file=True)

disk.plot(path=path, save_to_file=True)

In [ ]:
import Thompson_edited as edit

In [ ]:
MBH=10**5 * ct.MSun
# spin=0.9
# Rmin=jscript.R_isco_function(MBH, spin)
# R_G=MBH * ct.G /(ct.c*ct.c)
# Rout=1e7*R_G
# sigma = (200 * 1e3) * (MBH / (1.3e8*ct.MSun)) ** (1 / 4.24)
# Mdot_out = 320*(ct.MSun/ct.yr)*(Rout/(95*ct.pc)) * (sigma/(188e3))**2

R_G=MBH * ct.G /(ct.c*ct.c)
Rout=1e7*R_G
sigma = (200 * 1e3) * (MBH / (1.3e8*ct.MSun)) ** (1 / 4.24)
Mdot_out = 320*(ct.MSun/ct.yr)*(Rout/(95*ct.pc)) * (sigma/(188e3))**2
diskTQM = pagn.ThompsonAGN(Mbh=MBH, Mdot_out=Mdot_out)

# diskTQM = pagn.ThompsonAGN(Mbh=MBH, Rin=Rmin, Rout=Rout, Mdot_out=Mdot_out)
diskTQM.solve_disk()


In [ ]:
Mdotmaxs=[]
Mdotouts=[]
MBHs=[]

powers=np.linspace(4,8, 1000)

flag=0

for power in powers:
    print(power)
    MBH=10**power * ct.MSun
    MBHs.append(MBH)

    R_G=MBH * ct.G /(ct.c*ct.c)

    Rout=1e7*R_G

    sigma = (200 * 1e3) * (MBH / (1.3e8*ct.MSun)) ** (1 / 4.24)

    Mdot_out = 320*(ct.MSun/ct.yr)*(Rout/(95*ct.pc)) * (sigma/(188e3))**2

    print(Mdot_out)

    diskTQM = edit.ThompsonAGN(Mbh=MBH, Rin=Rmin, Rout=Rout, Mdot_out=Mdot_out)
    diskTQM.solve_disk()
    Mdotmax= diskTQM.Mdotmax * (ct.yr / ct.MSun)
    Mdotout= diskTQM.Mdot_out * (ct.yr / ct.MSun)
    Mdotmaxs.append(Mdotmax)
    Mdotouts.append(Mdotout)
    if Mdotmax>Mdotout and flag==0:
        power_crit=power
        flag+=1

print(f'Mdot max exceeds Mdot out at SMBH = 1e{power_crit} MSun')

plt.plot(powers, Mdotmaxs, color='r', label='$\dot{M}_{max}$')
plt.plot(powers, Mdotouts, color='b', label='$\dot{M}_{out}$')
plt.axvline(x=power_crit)
plt.xlabel('$log(M_{SMBH})$')
plt.ylabel('$\dot{M}~[M_{\odot}yr^{-1}]$')
plt.title("")
plt.legend()
plt.show()
# diskTQM.solve_disk(N=1e4)

In [ ]:
disk = pagn.SirkoAGN(Mbh=MBH, alpha=alpha, le= le, eps=eps)
Rmin = disk.Rmin
Rmax = disk.Rmax
disk.solve_disk()

In [ ]:
colour='plasma'
cmap = plt.colormaps[colour]

fig, axs = plt.subplots(1, 4, figsize=(20, 5), dpi=100)

i=0
axs[i].plot(disk.R/R_G, 2*disk.h*disk.rho*ct.SI_to_gcm2, label = r"SG", color=cmap(0.0))
axs[i].set_ylabel(r'$\Sigma_{\rm g} [{\rm g \, cm}^{-2}]$')
axs[i].set_xlabel("r/Rg")

# axs[i].set_xlim(5, 1e6)
# axs[i].set_ylim(1, 1e7)
axs[i].set_xscale('log')
axs[i].set_yscale('log')
axs[i].axvline(x=disk.R_AGN/R_G, linestyle='--', color=cmap(0.2), alpha=0.5, label = r"$R_{AGN}$")

axs[i].set_title(f'Surface Density vs Radius')

i=1
axs[i].plot(disk.R/R_G, disk.T, color=cmap(0.0), label = r"SG")
axs[i].set_ylabel(r'$T [K] $')
axs[i].set_xlabel("r/Rg")

axs[i].set_xscale('log')
axs[i].set_yscale('log')

# axs[i].set_xlim(5, 1e7)
# axs[i].set_ylim(1e1, 3e7)
axs[i].axvline(x=disk.R_AGN/R_G, linestyle='--', color=cmap(0.2), alpha=0.5, label = r"$R_{AGN}$")

axs[i].set_title(f'Temperature vs Radius')

i=2

axs[i].plot(disk.R/R_G, disk.h/disk.R, label = r"SG", color=cmap(0.0))
axs[i].set_ylabel(r'$h/r$')
axs[i].set_xlabel("r/Rg")

# axs[i].set_xlim(1, 1e7)
# axs[i].set_ylim(1, 1e7)
axs[i].set_xscale('log')
axs[i].set_yscale('log')
axs[i].axvline(x=disk.R_AGN/R_G, linestyle='--', color=cmap(0.2), alpha=0.5, label = r"$R_{AGN}$")

axs[i].set_title(f'Disc Thickness vs Radius')
axs[i].legend()

i=3

axs[i].plot(disk.R/R_G, disk.rho/1e3, label = r"SG", color=cmap(0.0))
axs[i].set_ylabel(r'$\rho [?]$')
axs[i].set_xlabel("r/Rg")

# axs[i].set_xlim(1, 1e7)
# axs[i].set_ylim(1, 1e7)
axs[i].set_xscale('log')
axs[i].set_yscale('log')
axs[i].axvline(x=disk.R_AGN/R_G, linestyle='--', color=cmap(0.2), alpha=0.5, label = r"$R_{AGN}$")

axs[i].set_title(f'Midplane Surface Density vs Radius')
axs[i].legend()


plt.tight_layout()
plt.savefig(f'EMRIs_Jupiter_2/SG_profiles_logMBH{MBH_power}_alpha{alpha}_3.pdf')
plt.show()

In [ ]:
def BHL_accretion(args, obj, MBH, mbh, Mdot, gamma):
    drhodr=jscript.drhodR(obj)
    
    h=obj.h
    cs=obj.cs
    r=obj.R
    rho=obj.rho
    sigma= 2*h*rho

    E=MBH/(2*r)
    J=MBH * np.sqrt(1/(2*E))

    dJdr=jscript.dJdR(obj, J)

    deltav_psi=h*cs*(3-drhodr)/2
    deltav_dr=3/2 * (mbh/(3*MBH))**(1/3) * (1/h) * cs
    deltav_r=np.abs(-Mdot/(2 * np.pi * r * sigma) - gamma/dJdr)
    vrel=((deltav_psi + deltav_dr)**2 + deltav_r**2)**(1/2)
    mdot_BHL= (4 * np.pi * rho * mbh * mbh) / (vrel**2 + cs**2)**(3/2)
    return mdot_BHL

In [ ]:
Mbh=10**4 * ct.MSun
alpha=0.1
mdot=0.1
eps=0.1
mbh=10 * ct.MSun

le=eps*mdot

M=Mbh * ct.G /(ct.c*ct.c)

Ledd=jscript.Ledd(Mbh, X=0.7)
Mdot=le * Ledd / eps

disk = pagn.SirkoAGN(Mbh=Mbh, alpha=alpha, le=le, eps=eps)
Rmin = disk.Rmin
Rmax = disk.Rmax
disk.solve_disk()

rho=disk.rho
cs=disk.cs
R=disk.R
h=disk.h
sigma=2*h*rho

gamma=jscript.drhodR(disk)

deltav_phi=(3-gamma)/2 * (h / R) * cs
deltav_dr=3/2 * (mbh/(3*Mbh))**(1/3) * (1/h) * cs

vgas=-Mdot/(2 * np.pi * R * sigma)
vstar=-1.3e-6 * (mbh/(10*ct.MSun))/(Mbh/(1e5*ct.MSun)) * (R/10*M)**(-3)
deltav_r=np.abs(vgas-vstar)

vrel_sqr=(deltav_phi + deltav_dr)**2 + deltav_r**2

mdot_BHL=(4 * np.pi * rho * mbh * mbh) / (vrel_sqr + cs**2)**(3/2)

print(mdot_BHL)

In [ ]:
from NT_disk_Eqns_V1 import A_fn, B_fn, C_fn, D_fn, E_fn, Q_fn

In [ ]:
def disc_solver(MBH_power, spin, alpha, mdot, eps, le, steps, path, printing=True, plotting=True, save_to_file=True):
    #general scaling from Abramowicz and Fragile
    MBH=10**MBH_power * ct.MSun #in kgs
    M=MBH * ct.G /(ct.c*ct.c)
    R_G=M

    m=MBH/ct.MSun
    Rsch= 2*ct.G*MBH/ct.c**2

    if mdot==None and le==None:
        raise ValueError('Please provide an accretion rate or Eddington ratio!')
    elif le==None:
        le=mdot*eps
    elif mdot==None:
        mdot=le/eps

    disk = pagn.SirkoAGN(Mbh=MBH, alpha=alpha, le=le, eps=eps)
    Rmin = disk.Rmin
    Rmax = disk.Rmax
    disk.solve_disk()

    print(f'SG: {Rmin/R_G} Rg, {disk.Rmin/R_G} Rg, {disk.R[0]/R_G} Rg')

    Rout=Rmax
    sigma = (200 * 1e3) * (MBH / (1.3e8*ct.MSun)) ** (1 / 4.24)
    Mdot_out = 320*(ct.MSun/ct.yr)*(Rout/(95*ct.pc)) * (sigma/(188e3))**2

    diskTQM = pagn.ThompsonAGN(Mbh=MBH, Rout=Rout, Rin=Rmin, Mdot_out=Mdot_out)
    diskTQM.solve_disk()

    print(f'TQM: {Rmin/R_G} Rg, {diskTQM.Rin/R_G} Rg, {diskTQM.R[0]/R_G} Rg')

    Rmin= jscript.R_isco_function(MBH, spin) #uses relativistic eqn for ISCO to set inner edge of disc
    r_isco=jscript.R_isco_function(MBH, spin)

    R=np.logspace(np.log10(r_isco), np.log10(Rmax), steps+1)
    # R=np.linspace(r_isco, Rmax, steps+1)

    R_im=jscript.R_inner_mid(r_isco, MBH, mdot, alpha)
    R_mo=jscript.R_mid_outer(r_isco, mdot)
    R_oa=jscript.R_outer_AGN(r_isco, MBH, mdot, alpha)

    R_agn=disk.R_AGN/R_G

    As=[]
    Bs=[]
    Cs=[]
    Ds=[]
    Es=[]
    Qs=[]

    coefficient=[]

    Rs=[]

    k=50
    if printing==True:
        print(R_oa/R_agn)
        print(f'Initial Radius = {R[k]/R_G} Rg, R_isco = {r_isco/R_G} Rg')

    for i in range(k, steps+1):
        r=R[i]
        y=np.sqrt(r/M)
        rstar=(r/M)
        r_im=jscript.r_in_mid(y, MBH, spin, mdot, alpha)
        r_mo=jscript.r_mid_outer(y, MBH, spin, mdot)

        A=A_fn(y, spin)
        B=B_fn(y, spin)
        C=C_fn(y, spin)
        D=D_fn(y, spin)
        E=E_fn(y, spin)
        Q=Q_fn(y, MBH, spin)

        coefficient.append(A**(2/3) * D**(-1/3) * Q**(2/3))

        As.append(A)
        Bs.append(B)
        Cs.append(C)
        Ds.append(D)
        Es.append(E)
        Qs.append(Q)

        Rs.append(rstar)
        
    plt.plot(Rs, As, label='A')
    # plt.plot(Rs, Bs, label='B')
    # plt.plot(Rs, Cs, label='C')
    plt.plot(Rs, Ds, label='D')
    # plt.plot(Rs, Es, label='E')
    plt.plot(Rs, Qs, label="Q")

    plt.plot(Rs, coefficient, label='coeff')

    plt.xlabel('R/M')

    plt.xscale('log')
    plt.yscale('log')

    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
mdot = 0.1 #in terms of Ledd
eps = 0.1
le = mdot*eps
# mdot=le/eps

disc_solver(4, 0.9, 0.1, mdot, eps, le, steps, path, printing=True, plotting=True, save_to_file=True)